In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import itertools
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.stattools import grangercausalitytests

In [ ]:
DISPOSAL_FILEPATH = "../../data/raw/council_disposal_receipts.xlsx"

YEAR_SKIP_MAPPING = {
    "2014": 0,
    "2015": 0,
    "2016": 2,
    "2017": 3,
    "2018": 3,
    "2019": 4,
    "2020": 4,
    "2021": 4,
    "2022": 4,
    "2023": 4,
}

dfs = []
for year, skip_row in YEAR_SKIP_MAPPING.items():
    df = pd.read_excel(DISPOSAL_FILEPATH, skiprows=skip_row, sheet_name=year)
    dfs.append(df)

dataset = pd.read_csv(r'../../data/processed/charity_main_cleaned.csv')

In [ ]:
def clean_sheet(df, rename_from_col=3, header_row=1):
    """
    Cleans a DataFrame read from Excel:
    - Replaces values in `header_row` with column names starting from `rename_from_col`
    - Keeps all columns up to `rename_from_col`
    - Keeps only renamed columns after `rename_from_col`
    - Sets row `header_row` as the header and drops all rows above it
    """
    # Step 1: Identify renamed columns
    renamed_mask = [not str(col).startswith("Unnamed:") for col in df.columns]

    # Step 2: Determine which columns to keep
    cols_to_keep = list(range(rename_from_col + 1))  # Always keep up to and including `rename_from_col`
    for i in range(rename_from_col + 1, len(df.columns)):
        if renamed_mask[i]:
            cols_to_keep.append(i)

    # Step 3: Overwrite header_row with column names for renamed columns
    for i in range(rename_from_col, len(df.columns)):
        if renamed_mask[i]:
            df.iloc[header_row, i] = df.columns[i]

    # Step 4: Set row `header_row` as header and drop rows above
    df.columns = df.iloc[header_row]
    df = df.drop(index=list(range(header_row + 1))).reset_index(drop=True)

    # Step 5: Keep only selected columns
    df = df.iloc[:, cols_to_keep]

    return df

def rename_and_filter_disposal(df, start_col=5, keyword=": Disposal of tangible fixed assets"):
    """
    Keeps:
    - All columns before `start_col`
    - Columns from `start_col` onward that contain `keyword`
    
    Renames matching columns by removing the keyword from their name.
    """
    cols_to_keep = list(range(start_col))  # Keep early columns as-is

    new_columns = df.columns.tolist()  # Copy of column names

    for i in range(start_col, len(df.columns)):
        col = str(df.columns[i])
        if keyword in col:
            # Rename column by removing the keyword
            new_columns[i] = col.replace(keyword, "")
            cols_to_keep.append(i)

    # Apply renaming
    df.columns = new_columns

    # Keep only selected columns
    return df.iloc[:, cols_to_keep]

dfs[0] = clean_sheet(dfs[0], rename_from_col=3, header_row=1)
dfs[1] = clean_sheet(dfs[1], rename_from_col=4, header_row=1)
dfs[2] = clean_sheet(dfs[2], rename_from_col=4, header_row=2)
dfs[3] = clean_sheet(dfs[3], rename_from_col=5, header_row=0)
dfs[4] = clean_sheet(dfs[4], rename_from_col=5, header_row=0)

dfs[5] = rename_and_filter_disposal(dfs[5], start_col=5)
dfs[6] = rename_and_filter_disposal(dfs[6], start_col=5)
dfs[7] = rename_and_filter_disposal(dfs[7], start_col=5)
dfs[8] = rename_and_filter_disposal(dfs[8], start_col=5)
dfs[9] = rename_and_filter_disposal(dfs[9], start_col=6)

In [ ]:
for i in range(10):
    df = dfs[i]

    # Step 1: Standardise column names first
    df.columns = df.columns.str.strip().str.lower()
    if 'la name' in df.columns:
        df = df.rename(columns={'la name': 'local_authority'})
    
    if 'local_authority' in df.columns:
        df['local_authority'] = df['local_authority'].str.replace(r'(?i)\(ua\)|\bua\b', '', regex=True).str.strip()

    # Step 2: Drop bad values in first column
    first_col = df.columns[0]
    df = df[~df[first_col].isin([pd.NA, None, '[z]', 'la_lgf_code'])]
    df = df.dropna(subset=[first_col])

    # Step 3: Drop missing ONS codes
    if i == 0 and 'ecode' in df.columns:
        df = df.dropna(subset=['ecode'])
    elif i != 0 and 'ons code' in df.columns:
        df = df.dropna(subset=['ons code'])

    # Step 4: Drop unwanted council classes
    if 'class' in df.columns:
        df = df[~df['class'].isin(['O', 'MC','SC'])]

    # Save cleaned frame back
    dfs[i] = df

In [ ]:
merge_columns = {
    "community safety": ["community safety", "community safety (cctv)"],
    'agricultural & fisheries services':['agriculture & fisheries'],
    'all services total':['total all services'],
    'early years & primary schools':['pre-primary & primary education'],
    'parking':['parking of vehicles'],
    'ports & piers':['local authority ports & piers'],
    'special schools & alternative provision':['special education'],
    'secondary schools':['secondary education'],
    'total industrial & commercial trading':['industrial & commercial trading'],
    'total environmental & regulatory services':['total environmental services','regulatory services (environmental health)'],
    'tolled roads, bridges, tunnels,ferries & public transport companies':['tolled road bridges, tunnels, ferries, public transport companies','tolled road bridges, tunnels, ferries & public transport companies'],
    'public transport (bus)':['public passenger transport - bus'],
    'public transport (rail & other)':['public passenger transport - rail & other'],
    'total housing':['housing'],
    'total police':['police'],
    'total social care':['social services','social care'],
    'total public health':['public health'],
    'roads, street lighting & road safety':['roads, street lights & safety'],
    'total fire & rescue services':['fire & rescue services'],
    'total central services':['central services (including court services)'],
    'street cleaning (not chargeable to highways)':['street cleaning not chargeable to highways'],
    'total planning & development':['total planning & development services','planning & development services'],
    'total trading services':['total trading','trading'],
    'total education':['education'],
    'total highways & transport':['total transport','highways & transport'],
    'total culture & related services':['culture & heritage']

    #'commercial housing',
    #'energy generation & supply',
    #'finance & insurance activity',
    #'hospitality & catering',
    #'lgf code',
    #'ons code',
    #'other commercial activity',
    #'other real estate activities',
    #'post-16 provision & other education',
    #'subclass',
    #'total digital infrastructure',
    #'water supply, sewerage & remediation'
}

# Standardise and merge alias columns across all dfs
for idx in range(len(dfs)):
    df = dfs[idx].copy()
    df.columns = [
        str(col).strip().lower()
        .replace(" and ", " & ")
        for col in df.columns
    ]

    dfs[idx] = df  # Save the cleaned version

    for standard_col, aliases in merge_columns.items():
        # Find which alias columns are present
        present_cols = [col for col in aliases if col in df.columns]
        if not present_cols:
            continue  # Nothing to merge for this category

        # Sum across all present columns
        df[standard_col] = df[present_cols].sum(axis=1, skipna=True)

        # Drop duplicates, except the one we’re keeping
        cols_to_drop = [col for col in present_cols if col != standard_col]
        df.drop(columns=cols_to_drop, inplace=True)

    dfs[idx] = df  # Save the cleaned DataFrame back

In [ ]:
# Combined mapping: new_name → list of old names
unified_map = {
    'buckinghamshire': ['aylesbury vale', 'chiltern', 'south bucks', 'wycombe','south buckinghamshire'],
    'dorset': ['weymouth and portland', 'west dorset', 'north dorset', 'east dorset', 'purbeck', 'christchurch'],
    'somerset': ['taunton deane', 'west somerset', 'mendip', 'sedgemoor', 'south somerset', 'somerset council','somerset west and taunton'],
    'cumberland': ['allerdale', 'carlisle', 'copeland', 'cumberland council'],
    'westmorland and furness': ['barrow in furness', 'barrow-in-furness', 'eden', 'south lakeland'],
    'north yorkshire': ['craven', 'hambleton', 'harrogate', 'richmondshire', 'ryedale', 'scarborough', 'selby', 'north yorkshire council'],
    'bournemouth christchurch and poole': ['bournemouth', 'christchurch', 'poole'],
    'west suffolk': ['forest heath', 'st edmundsbury'],
    'east suffolk': ['suffolk coastal', 'waveney'],
    'bath and north east somerset': ['bath and ne somerset'],
    'southend-on-sea': ['southend on sea'],
    'leicester': ['leicester city'],
    'medway': ['medway towns'],
    'derby': ['derby city'],
    'folkestone and hythe': ['shepway'],
    'county durham': ['durham'],
    "king's lynn and west norfolk": ['kings lynn and west norfolk'],
    'north northamptonshire': ['wellingborough', 'east northamptonshire', 'kettering', 'corby'],
    'west northamptonshire': ['northampton', 'south northamptonshire', 'daventry'],
}
# old_name → new_name (lowercase)
flat_lookup = {
    old.lower(): new.lower()
    for new, olds in unified_map.items()
    for old in olds
}
def clean_local_authority(name):
    if pd.isna(name):
        return name
    name = str(name).lower()
    name = (
        name.replace('&', 'and')
        .replace('-', ' ')
        .replace(',', '')
        .replace('.', '')
        .replace(' city of', '')
        .replace(' county of', '')
        .strip()
    )
    return flat_lookup.get(name, name)
# Apply to disposal files
for i in range(len(dfs)):
    if 'local_authority' in dfs[i].columns:
        dfs[i]['local_authority'] = dfs[i]['local_authority'].apply(clean_local_authority)

# Apply to charity dataset
if 'local_authority' in dataset.columns:
    dataset['local_authority'] = dataset['local_authority'].apply(clean_local_authority)

# List of known devolved nation council names to exclude
non_england_keywords = [
    'aberdeen', 'aberdeenshire', 'angus', 'antrim', 'ards', 'argyll', 'armagh', 'belfast', 'blaenau', 'bridgend',
    'caerphilly', 'cardiff', 'carmarthenshire', 'causeway', 'ceredigion', 'conwy', 'denbighshire', 'derry',
    'dumfries', 'dundee', 'east ayrshire', 'east dunbartonshire', 'east lothian', 'east renfrewshire', 'falkirk',
    'fermanagh', 'fife', 'flintshire', 'glasgow', 'gwynedd', 'highland', 'inverclyde', 'isle of man',
    'isle of anglesey', 'lisburn', 'merthyr', 'mid and east antrim', 'mid ulster', 'midlothian', 'monmouthshire',
    'moray', 'na h eileanan siar', 'neath', 'newport', 'newry', 'north ayrshire', 'north lanarkshire', 'orkney',
    'pembrokeshire', 'perth and kinross', 'powys', 'renfrewshire', 'rhondda', 'scottish borders', 'shetland',
    'south ayrshire', 'south lanarkshire', 'stirling', 'swansea', 'torfaen', 'vale of glamorgan', 'west dunbartonshire',
    'west lothian', 'wrexham','city of edinburgh','channel islands'
]

# Convert to lowercase and filter out rows containing these names
dataset['local_authority_lower'] = dataset['local_authority'].str.lower()
dataset = dataset[~dataset['local_authority_lower'].str.contains('|'.join(non_england_keywords), na=False)]

# Drop the helper column
dataset = dataset.drop(columns=['local_authority_lower'])

In [ ]:
# Step 1: Get unique local_authority names from dfs[0–9]
all_disposal_names = set()
for i in range(10):
    if 'local_authority' in dfs[i].columns:
        all_disposal_names.update(dfs[i]['local_authority'].dropna().unique())

# Step 2: Get unique local_authority names from dataset
if 'local_authority' in dataset.columns:
    charity_names = set(dataset['local_authority'].dropna().unique())
else:
    raise KeyError("'local_authority' column not found in dataset")

# Step 3: Find mismatches
in_disposal_not_in_charity = all_disposal_names - charity_names
in_charity_not_in_disposal = charity_names - all_disposal_names

# Step 4: Print results
print("✅ In disposal (dfs[0–9]) but NOT in charity dataset:")
print(sorted(in_disposal_not_in_charity))

print("\n🔍 In charity dataset but NOT in disposal (dfs[0–9]):")
print(sorted(in_charity_not_in_disposal))


In [ ]:
# Step 1: Assign financial year to each DataFrame in dfs
years = list(range(2014, 2024))
for i, year in enumerate(years):
    if not dfs[i].empty:
        dfs[i]['financial_year'] = year

# Step 2: Melt each DataFrame into long format
long_frames = []
for i, df in enumerate(dfs):
    if df.empty:
        continue
    long_df = df.melt(
        id_vars=["local_authority", "financial_year"],
        value_vars=[
            col for col in df.columns 
            if col not in ["ecode", 'lgf code', 'ons code', "class", "subclass", 
                           "local_authority", "financial_year", 'certification complete']
        ],
        var_name="category",
        value_name="value"
    )
    long_frames.append(long_df)

# Step 3: Combine all disposal long-format DataFrames
disposal_long_df = pd.concat(long_frames, ignore_index=True)

# Step 4: Count removals from charity dataset
removals = (
    dataset
    .groupby(['local_authority', 'removal_fy', 'size_category'])
    .size()
    .reset_index(name='removals')
    .rename(columns={'removal_fy': 'financial_year'})
)

# Step 5: Merge disposal data with charity removals
panel = pd.merge(
    disposal_long_df,
    removals,
    on=["financial_year", "local_authority"],
    how="outer"
)

# Step 6: Replace NaNs in 'removals' with 0
panel['removals'] = panel['removals'].fillna(0).astype(int)
panel = panel[(panel['financial_year'] >= 2015) & (panel['financial_year'] <= 2023)]
panel['value'] = pd.to_numeric(panel['value'], errors='coerce') / 1000000


In [ ]:
# Step 1: Separate disposal data (no size_category)
disposal = panel[['local_authority', 'financial_year', 'category', 'value']].drop_duplicates()

# Step 2: Separate charity data (includes size_category)
charity = panel[['local_authority', 'financial_year', 'category', 'size_category', 'removals']]

# Step 3: Generate full index for all combinations
all_authorities = panel['local_authority'].unique()
all_years = panel['financial_year'].unique()
all_sizes = ['Small', 'Medium', 'Large']
all_categories = panel['category'].unique()

full_index = pd.DataFrame(
    list(itertools.product(all_authorities, all_years, all_categories, all_sizes)),
    columns=['local_authority', 'financial_year', 'category', 'size_category']
)

# Step 4: Merge disposal (no size_category, shared across sizes)
merged = pd.merge(
    full_index,
    disposal,
    on=['local_authority', 'financial_year', 'category'],
    how='left'
)

# Step 5: Merge removals
merged = pd.merge(
    merged,
    charity,
    on=['local_authority', 'financial_year', 'category', 'size_category'],
    how='left'
)

# Step 6: Fill missing values
merged['value'] = pd.to_numeric(merged['value'], errors='coerce').fillna(0)
merged['removals'] = merged['removals'].fillna(0).astype(int)

# Resulting panel
panel_complete = merged
panel_complete

In [ ]:
# Resulting filtered panel
filtered_panel = panel_complete[panel_complete['category']=='all services total']
# Ensure proper sorting
filtered_panel = filtered_panel.sort_values(by=['local_authority', 'size_category', 'financial_year'])

# Create lagged value columns
filtered_panel['value_lag1'] = (
    filtered_panel
    .groupby(['local_authority', 'size_category'])['value']
    .shift(1)
)

filtered_panel['value_lag2'] = (
    filtered_panel
    .groupby(['local_authority', 'size_category'])['value']
    .shift(2)
)

filtered_panel['value_lag3'] = (
    filtered_panel
    .groupby(['local_authority', 'size_category'])['value']
    .shift(3)
)


In [ ]:
# ADF test on 'removals'
result_removals = adfuller(filtered_panel['removals'].dropna())
print("ADF Test for 'removals':")
print(f"ADF Statistic: {result_removals[0]}")
print(f"p-value: {result_removals[1]}")
print(f"Critical Values: {result_removals[4]}\n")

# ADF test on 'value'
result_value = adfuller(filtered_panel['value'].dropna())
print("ADF Test for 'value':")
print(f"ADF Statistic: {result_value[0]}")
print(f"p-value: {result_value[1]}")
print(f"Critical Values: {result_value[4]}")

In [ ]:
# Run Engle-Granger cointegration test
coint_stat, p_value, crit_values = coint(filtered_panel['removals'], filtered_panel['value'])

print(f"Engle-Granger Cointegration Test")
print(f"Test Statistic: {coint_stat}")
print(f"p-value: {p_value}")
print(f"Critical Values: {crit_values}")

In [ ]:
# Format data as a two-column array: [removals, value]
data = filtered_panel[['removals', 'value']].dropna()

# Run Granger causality test with up to 3 lags
grangercausalitytests(data, maxlag=3)


In [ ]:
# Prepare data: ensure it's sorted and aligned
df = filtered_panel[['removals', 'value']].dropna().copy()
df = df.astype(float)

# Run Granger causality test: does 'removals' Granger-cause 'value'?
# maxlag = 3 for testing 1, 2, 3 lags
reverse_granger_results = grangercausalitytests(df[['value', 'removals']], maxlag=3, verbose=True)


In [ ]:
# Run OLS regression with fixed effects for LA and year
model = smf.ols(
    formula='removals ~ value + C(local_authority) + C(financial_year) + C(size_category)+value:C(size_category)',
    data=filtered_panel
).fit()

print(model.summary())

In [ ]:
model = smf.ols(
    formula=(
        'removals ~ '
        'value + value_lag1 + value_lag2 + value_lag3 + '
        'C(local_authority) + C(financial_year) + C(size_category) + '
        'value:C(size_category) + '
        'value_lag1:C(size_category) + '
        'value_lag2:C(size_category) + '
        'value_lag3:C(size_category)'
    ),
    data=filtered_panel
).fit()

print(model.summary())


In [ ]:
filtered_panel_1 = panel_complete[
    panel_complete['category'].str.startswith('total') &
    ~panel_complete['category'].str.contains(', of which')
]

In [ ]:
panel_complete['category'].unique()

In [ ]:
filtered_panel_2 = panel_complete[
    panel_complete['category'].isin([
        'total housing',
        'total environmental & regulatory services',
        'total culture & related services',
        'tourism'
    ])
]


In [ ]:
model = smf.ols(
    formula=(
        'removals ~ value + C(local_authority) + C(financial_year) + C(category) + C(size_category) + value:C(size_category) + value:C(category)'
    ),
    data=filtered_panel_1
).fit()

print(model.summary())


In [ ]:
# For example, only on 'total housing'
housing_panel = panel_complete[panel_complete['category'] == 'total culture & related services']
model = smf.ols('removals ~ value + C(local_authority) + C(financial_year) + C(size_category)', data=housing_panel).fit()
print(model.summary())


In [ ]:
# Run OLS regression with fixed effects for LA and year
model = smf.ols(
    formula='value ~ removals + C(local_authority) + C(financial_year) + C(category) + C(size_category)+removals:C(size_category) + removals:C(category)',
    data=filtered_panel_2
).fit()

print(model.summary())